In [58]:
import torch

torch.manual_seed(11)

letters = ['h','i', 'e', 'l', 'o']

# hihell -> ihello
x_data = [[0, 1, 0, 2, 3, 3]]
y_data = [1, 0, 2, 3, 3, 4]

inputs = torch.LongTensor(x_data)
labels = torch.LongTensor(y_data)

print(inputs)
print(labels)

tensor([[0, 1, 0, 2, 3, 3]])
tensor([1, 0, 2, 3, 3, 4])


In [59]:
import torch.nn as nn

class_n = 5
input_size = 5
hidden_size = 5
batch_size = 1
embedding_size = 10
seq_n = 6
layer_n = 1

class Model(nn.Module):
    
    def __init__(self):
        super(Model, self).__init__()
        
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.RNN(input_size = embedding_size, hidden_size = 5,
                         batch_first = True)
        
        self.fc = nn.Linear(hidden_size, class_n) # 뭔진 아는데, 이게 왜 있지?
        
    def forward(self, x):
        
        h0 = torch.zeros(layer_n, batch_size, hidden_size)
        emb = self.embedding(x)
        emb = emb.view(batch_size, seq_n, -1)
        
        output, _ = self.rnn(emb, h0)
        output = self.fc(output.view(-1, class_n))
        
        return output

model = Model()
print(model)

Model(
  (embedding): Embedding(5, 10)
  (rnn): RNN(10, 5, batch_first=True)
  (fc): Linear(in_features=5, out_features=5, bias=True)
)


In [60]:
for p in model.parameters():
    print(p) # input -> emb lookup table / emb -> hidden / hidden -> output / output -> output

Parameter containing:
tensor([[-0.5108,  1.0283, -0.3532,  0.1230, -0.1816, -1.4972,  0.1421, -0.5243,
         -0.2487, -0.5252],
        [ 2.8922, -0.5947,  1.3118,  0.3522, -1.3151, -0.0080,  0.2479,  1.5727,
         -1.6395, -1.5925],
        [-0.1546, -1.0964,  1.3666,  0.6893, -0.3935,  0.6171,  0.7528,  0.6023,
          2.0175, -1.1686],
        [-1.3242,  1.1267, -0.2255,  0.5218,  0.5485, -1.5885,  0.5281,  0.8964,
          0.9975, -0.2156],
        [ 0.0927,  0.3910, -1.0265,  0.0436, -0.9724,  0.2803,  0.5700,  1.4841,
         -0.7572, -1.1598]], requires_grad=True)
Parameter containing:
tensor([[-0.3324,  0.1679, -0.0803,  0.1296,  0.4206, -0.4330,  0.0790,  0.2685,
          0.0374, -0.2677],
        [-0.0824, -0.1333,  0.3466,  0.3995, -0.1756, -0.4392, -0.1417, -0.2468,
          0.3809,  0.4222],
        [ 0.2309, -0.0382, -0.4312,  0.0321, -0.1722,  0.0050, -0.2094, -0.0466,
          0.2527,  0.4416],
        [ 0.3677, -0.2432,  0.2912,  0.2909,  0.1795,  0.4241, 

In [61]:
import torch.optim as optim

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.1)

epoch_n = 100

for epoch in range(1, epoch_n + 1):
    optimizer.zero_grad()
    loss = 0
    
    output = model(inputs)
    loss = loss_function(output, labels)
    loss.backward()
    optimizer.step()
    
    val, idx = output.max(1)
    for i in idx:
        print(letters[i], end = '')
        
    print(' Epoch : %d/100, Loss = %1.3f' %(epoch, loss))

hohhhh Epoch : 1/100, Loss = 1.833
oholhh Epoch : 2/100, Loss = 1.391
ehelll Epoch : 3/100, Loss = 1.175
ehelll Epoch : 4/100, Loss = 1.021
ihilll Epoch : 5/100, Loss = 0.858
ihilll Epoch : 6/100, Loss = 0.738
ihelll Epoch : 7/100, Loss = 0.657
ehelll Epoch : 8/100, Loss = 0.599
ehello Epoch : 9/100, Loss = 0.555
ehello Epoch : 10/100, Loss = 0.521
ihillo Epoch : 11/100, Loss = 0.493
ihillo Epoch : 12/100, Loss = 0.466
ihillo Epoch : 13/100, Loss = 0.439
ihello Epoch : 14/100, Loss = 0.414
ehello Epoch : 15/100, Loss = 0.393
ehello Epoch : 16/100, Loss = 0.373
ehello Epoch : 17/100, Loss = 0.355
ihillo Epoch : 18/100, Loss = 0.339
ihillo Epoch : 19/100, Loss = 0.325
ihillo Epoch : 20/100, Loss = 0.313
ihillo Epoch : 21/100, Loss = 0.303
ehello Epoch : 22/100, Loss = 0.294
ehello Epoch : 23/100, Loss = 0.287
ehello Epoch : 24/100, Loss = 0.281
ihillo Epoch : 25/100, Loss = 0.275
ihillo Epoch : 26/100, Loss = 0.271
ihillo Epoch : 27/100, Loss = 0.267
ihillo Epoch : 28/100, Loss = 0.263
e

# 새로 알게 된 것

### 1. nn.Embedding(embedding_n, embedding_dim)<br>
Look up table, 말 그대로 Tensor에 있는 element를 dim에 맞게 임베딩하고 해당 table을 만들어줌. <br>그냥 index를 설정해준다고 생각하면 된다.
- 임베딩 할 요소의 개수(embedding_n)
- 임베딩할 때 차원의 수(embedding_dim)

### 2. Embedding을 하는 이유
- 일단 벡터로 표현된 단어들은 서로 배타적이지 않다. 즉, 연관이 되어있기 때문에 그 이질성을 최소화하기 위해
- 또한 원-핫 벡터의 경우 차원의 크기가 기하급수적으로 커지니 차원을 축소하기 위해 임베딩을 한다.
- https://www.quora.com/What-does-PyTorch-Embedding-do